# Exploration band power from David's pipline

In [1]:
import os 
import pickle
import  json
import re
project_path = '/scratch7/MINDLAB2021_MEG-TempSeqAges/learning_bach_iEEG'
scripts_path = os.path.join(project_path, 'scripts')
from sys import path; path.append(scripts_path)

event_id = {'old/correct': 1,
 'new/correct': 2,
 'old/incorrect': 101,
 'new/incorrect': 102,
 'old/null': 201,
 'new/null': 202}

import numpy as np

# fack cgi
import types, sys
sys.modules['cgi'] = types.ModuleType('cgi')

task = 'MusicMemory'

band_power_path = project_path + '/data/band_power'
freq_band = ['delta', 'theta','alpha', 'low_beta', 'high_beta', 'low_gamma', 'high_gamma']
out_path = '/users/barbara/Desktop/MIB/outs/TFR'

In [2]:
# TFRm
summary = True 
for subj in os.listdir(band_power_path) :
    tfr_list = []
    info = {}
    files_freq_band =  [band_power_path + '/' + subj + '/TFR/' + subj + '_band_power_LB_' + f + '_25_100.p' for f in freq_band]
    for file in files_freq_band : 
        with open(file, "rb") as f:
            TFRm = pickle.load(f)
        TFRm.crop(tmin=-0.05, tmax=3.95)

        unique_ev = np.unique(TFRm.events[:, 2])
        TFRm_ = TFRm.get_data()
        id_ev ={}

        info['chnames'] = TFRm.info['ch_names']
        info['time'] = list(TFRm.times)
        info['event_count'] = [[list(event_id.keys())[list(event_id.values()).index(event)], str(count)] for event, count in zip(np.unique(TFRm.events[: , 2], return_counts=True)[0], np.unique(TFRm.events[: , 2], return_counts=True)[1])]

        for val in unique_ev :
            idx = np.where(TFRm.events[:, 2] == val)[0]
            id_ev[val] = idx

        TFR_mean_band_trials = np.zeros((len(event_id.values()), TFRm_.shape[1], TFRm_.shape[2]))
        for val, idx in id_ev.items():
            ev_id = list(event_id.values()).index(val)
            TFR_mean_band_trials[ev_id, :, :] = np.mean(TFRm_[idx, :, :], axis=0)
        
        tfr_list.append(TFR_mean_band_trials)
    data = np.stack(tfr_list)
    data_reshaped = np.transpose(data, (1, 2, 0, 3))
    out_file = out_path + f'/{subj}_TFRm.p'
        
    #with open(out_file, "wb") as f:
        #pickle.dump(data_reshaped, f)

    #with open(out_path + f'/{subj}_info.json', "w") as f:
    #    json.dump(info, f)

In [12]:
len(TFRm.ch_names)

58

In [13]:
len(TFRm.get_montage().ch_names)
 #https://mne.tools/stable/auto_tutorials/clinical/20_seeg.html#sphx-glr-auto-tutorials-clinical-20-seeg-py

58

In [4]:
for subj in os.listdir(band_power_path)[:2]:
    tfr_list = []
    info = {}
    files_freq_band =  [band_power_path + '/' + subj + '/TFR/' + subj + '_band_power_LB_' + f + '_25_100.p' for f in freq_band]
    for file in files_freq_band : 
        with open(file, "rb") as f:
            TFRm = pickle.load(f)
        TFRm.crop(tmin=-0.05, tmax=3.95)

In [5]:
id_ev1 = np.where(TFRm.events[:, 2] == 1)[0]
id_ev2 = np.where(TFRm.events[:, 2] == 2)[0]

In [29]:
# Raw with trials
#test_sample = {}
truth = {}

for subj in os.listdir(band_power_path):
    tfr_list = []
    info = {}
    files_freq_band =  [band_power_path + '/' + subj + '/TFR/' + subj + '_band_power_LB_' + f + '_25_100.p' for f in freq_band]
    for file in files_freq_band : 
        with open(file, "rb") as f:
            TFRm = pickle.load(f)
        TFRm.crop(tmin=-0.05, tmax=3.95)

        TFRm_ = TFRm.get_data()

        id_ev1 = np.where(TFRm.events[:, 2] == 1)[0]
        id_ev2 = np.where(TFRm.events[:, 2] == 2)[0]

        # Keep for test 
        #rnd_1 = np.random.choice(id_ev1)
        #rnd_2 = np.random.choice(id_ev2)
        #id_ev1 = id_ev1[id_ev1 != rnd_1]
        #id_ev2 = id_ev2[id_ev2 != rnd_2]
        id_ev = np.concatenate([id_ev1, id_ev2])

        # full size 
        TFR_trials = np.full((24, 2, TFRm_.shape[1], TFRm_.shape[2]), np.nan)
        TFR_trials[:len(id_ev1), 0, :, :] =  TFRm_[id_ev1, :, :]
        TFR_trials[:len(id_ev2), 1, :, :] =  TFRm_[id_ev2, :, :]

        #TFR_trials_test = TFRm_[[rnd_1] + [rnd_2], :, :]

        # mean to fill -- data augmentation
        event_means = np.nanmean(TFRm_[id_ev, :, :], axis=0, keepdims=True)
        nan_mask = np.isnan(TFR_trials)
        TFR_trials_filled = np.where(nan_mask, event_means, TFR_trials)

        # keep track of the ones augmented 
        true_trials = ~np.any(np.isnan(TFR_trials), axis=(2, 3))

        # save path
        out_path = f'outs/TFR/{subj}_trials'
        band = re.search(r'LB_([a-zA-Z_]+)_\d+_\d+\.p', file).group(1)
        out_file = f'{out_path}/{subj}_trials_{band}.p'

        if not os.path.exists(out_path) : 
            os.makedirs(out_path)
            
        with open(out_file, "wb") as f:
            pickle.dump(TFR_trials_filled, f)

        #with open(f'{out_path}/{subj}_trials_ev1_{band}.p', "wb") as f:
        #    pickle.dump(TFRm_[id_ev1[id_ev1 != rnd_1], :, :], f)

        #with open(f'{out_path}/{subj}_trials_ev2_{band}.p', "wb") as f:
        #    pickle.dump(TFRm_[id_ev2[id_ev2 != rnd_2], :, :], f)

        # test sample
        try : 
            truth[band].append(true_trials)
        except : 
            truth[band] = []
            truth[band].append(true_trials)
           

for band in list(truth.keys()) : 
    Truth_all = np.stack(truth[band], axis=2)*1

    with open(f'outs/TFR/True_trials_{band}.p', "wb") as f:
            pickle.dump(Truth_all, f)


In [62]:
with open(out_file, "rb") as f:
    TFR_trials = pickle.load(f)

In [5]:
# Check the coords in TFRm and info
subj = 'BJH026' 
files_freq_band =  [band_power_path + '/' + subj + '/TFR/' + subj + '_band_power_LB_' + f + '_25_100.p' for f in freq_band]
for file in files_freq_band[:1] : 
    with open(file, "rb") as f:
        TFRm_band = pickle.load(f)

# elect
import pandas as pd
elec_file = '/users/barbara/Desktop/MINDLAB2021_MEG-TempSeqAges/scratch/learning_bach_iEEG/misc/electrodes/BJH026_electrodes.csv'
elec = pd.read_csv(elec_file).set_index('electrode')
        
with open(f'{out_path}/{subj}_info.json') as json_data:
    d = json.load(json_data)
    time = d['time']
    json_data.close()

In [35]:
raw.annotations.description

NameError: name 'raw' is not defined

In [24]:
# Ectract the coords from the TFR mne object 
df = pd.DataFrame(columns=["x", "y", "z", 'subj','ch']).set_index(['subj','ch'])

for channel in range(len(TFRm_band.info['dig'])): 
    s = str(TFRm_band.info['dig'][channel])
    coords = re.findall(r"-?\d+\.?\d*", s)
    lab, x, y, z = map(float, coords)
    df.loc[(subj, int(lab)), : ] = [x, y, z]
df


x     y     z
subj   ch                   
BJH026 1    -4.9  27.3 -15.0
       2    -8.8  27.5 -14.0
       3   -12.7  27.8 -12.9
       4   -16.1  28.0 -11.9
       5   -20.0  28.3 -10.9
...          ...   ...   ...
       133  52.1 -10.5 -19.5
       134  56.1 -11.2 -19.9
       135  60.0 -11.8 -20.2
       136  63.8 -12.4 -20.0
       137  68.2 -12.4 -20.0

[137 rows x 3 columns]

In [23]:

lab, x, y, z = map(float, coords)
df = pd.DataFrame(columns=["x", "y", "z", 'subj','ch']).set_index(['subj','ch'])
df.loc[(subj, int(lab)), : ] = [x, y, z]
df

,,x,y,z
subj,ch,,,
BJH026,1,-4.9,27.3,-15.0


In [10]:
for ch, ch_i in zip(TFRm_band.info['ch_names'], d['chnames']) : 
    print(ch, ch_i)

AL1 AL1
AL2 AL2
AL3 AL3
AL5 AL5
AL6 AL6
AL8 AL8
AL9 AL9
AL10 AL10
AL11 AL11
AL12 AL12
AL13 AL13
AL14 AL14
BL1 BL1
BL2 BL2
BL3 BL3
BL4 BL4
BL5 BL5
BL6 BL6
BL7 BL7
BL8 BL8
BL9 BL9
BL11 BL11
BL12 BL12
BL13 BL13
BL14 BL14
CL1 CL1
CL2 CL2
CL3 CL3
CL4 CL4
CL5 CL5
CL6 CL6
CL7 CL7
DL1 DL1
DL2 DL2
DL3 DL3
DL4 DL4
DL10 DL10
DL11 DL11
DL12 DL12
DL13 DL13
EL1 EL1
EL2 EL2
EL3 EL3
EL4 EL4
EL5 EL5
EL9 EL9
FL1 FL1
FL2 FL2
FL3 FL3
FL4 FL4
FL5 FL5
FL6 FL6
FL7 FL7
FL8 FL8
FL9 FL9
IL11 IL11
JL5 JL5
JL6 JL6
JL7 JL7
JL8 JL8
JL9 JL9
JL10 JL10
KL1 KL1
KL2 KL2
KL3 KL3
KL4 KL4
KL5 KL5
KL10 KL10
KL11 KL11
LL1 LL1
LL2 LL2
LL3 LL3
LL4 LL4
LL5 LL5
LL6 LL6
LL7 LL7
GR1 GR1
GR2 GR2
GR3 GR3
GR4 GR4
GR5 GR5
GR9 GR9
GR10 GR10
